Monografia - Parte 2

Especialización en Analítica y Ciencia de Datos

Universidad de Antioquia

Grupo 3. Melissa Ortega Alzate CC.1036964792

# Objetivo

# Resumen

# Librerías

In [1]:
# Data treatment
import numpy as np
import pandas as pd

# Figures
import matplotlib.pyplot as plt
import seaborn as sns

# Data preprocessing
from sklearn.model_selection import train_test_split

# Librerías de Deep Learning
from keras import layers, models

# Models metrics
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

# Warnings configuration
import warnings
warnings.filterwarnings('ignore')

2024-03-28 19:29:23.101515: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-28 19:29:23.172494: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-28 19:29:23.477456: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-28 19:29:23.477523: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-28 19:29:23.529082: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

# Funciones

In [2]:
# ROC curve function

custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.set_theme( palette= "pastel", style= "ticks", rc= custom_params)

def plot_roc_curve(models: list, X_test, y_test):
    models_name = []
    for model in models:
        models_name.append(f'{model}'.split('(')[0])

    fig = plt.figure(figsize=(10, 6))
    entries = []

    for i in range(len(models)):
        model = models[i]
        probs = model.predict_proba(X_test)              # Obtener probabilidades de predicción del modelo
        probs = probs[:, 1]                              # Seleccionar la columna correspondiente a la probabilidad de clase positiva
        auc = roc_auc_score(y_test, probs)               # Calcular el AUC-ROC
        fpr, tpr, _ = roc_curve(y_test, probs)           # Calcular las tasas de FPR y TPR      
        plt.plot(fpr, tpr, label=f"{model.__class__.__name__} (AUC={auc:.2f})") # Graficar la curva ROC del modelo actual
        entries.append(( models_name[i], auc))
        #print("{:.2f}: {}".format(auc, model))            # Mostrar el AUC-ROC y el nombre del modelo

    df_roc = pd.DataFrame(entries, columns=['Model', 'AUC'])
    print(f" AUC de los modelos con orden descendente " +"\n" + "="*45)
    pd.options.display.max_colwidth = 100
    display(df_roc.sort_values(by='AUC', ascending= False))
    print("\n")

    plt.plot([0, 1], [0, 1], linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend()
    plt.show();

# Desarrollo

### Cargar datos

In [3]:
# Cargar X desde el archivo .npy
X = np.load('X.npy')
X[0]

array([0.36078431, 0.37254902, 0.39215686, ..., 0.57254902, 0.45098039,
       0.32941176])

In [4]:
# Cargar y desde el archivo .csv
y = pd.read_csv('y.csv')['Salmonella']
y[0]

1

### Split train and test

In [5]:
# Dividir el DataFrame en los trenes de entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(X, y,  
                                                    train_size= 0.8, 
                                                    shuffle= True, 
                                                    random_state= 123, 
                                                    stratify= y)

# Imprimir el tamaño de los trenes de entrenamiento y prueba
print("Tamaño del tren de entrenamiento:", X_train.shape)
print("Tamaño de las etiquetas de entrenamiento:", y_train.shape)
print("-------")
print("Tamaño del tren de prueba:", X_test.shape)
print("Tamaño de las etiquetas de prueba:", y_test.shape)

Tamaño del tren de entrenamiento: (800, 150528)
Tamaño de las etiquetas de entrenamiento: (800,)
-------
Tamaño del tren de prueba: (200, 150528)
Tamaño de las etiquetas de prueba: (200,)


### Baseline

In [6]:
# Instanciar el modelo de tipo Sequential
model = models.Sequential()

# 2. Capa densa de activación con 512 neuronas y una activación de tipo relu
model.add(layers.Dense(512, activation='relu', input_shape = (224*224*3,)))

# 3. Capa adicional con 1 neuronas (porque tengo 1 posible salida) y una activación tipo sigmoid para clasificación binara
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()

2024-03-28 19:29:40.901610: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-28 19:29:40.902378: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               77070848  
                                                                 
 dense_1 (Dense)             (None, 1)                 513       
                                                                 
Total params: 77071361 (294.00 MB)
Trainable params: 77071361 (294.00 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [7]:
# Compilar el modelo
model.compile(loss='binary_crossentropy',
            optimizer = 'rmsprop',
            metrics ='accuracy')

In [8]:
# Cambiar a 2 dimensiones
X_train = X_train.reshape(X_train.shape[0], 224*224*3)      
X_test = X_test.reshape(X_test.shape[0], 224*224*3)      

In [9]:
# Entrenar el modelo
model.fit(X_train, y_train, batch_size= 100, epochs=5)

Epoch 1/5
8/8 [==============================] - 8s 878ms/step - loss: 297.1038 - accuracy: 0.5063
Epoch 2/5
8/8 [==============================] - 7s 877ms/step - loss: 154.9640 - accuracy: 0.5088
Epoch 3/5
8/8 [==============================] - 7s 838ms/step - loss: 125.7963 - accuracy: 0.5050
Epoch 4/5
8/8 [==============================] - 6s 820ms/step - loss: 120.9531 - accuracy: 0.4688
Epoch 5/5
8/8 [==============================] - 6s 769ms/step - loss: 100.7939 - accuracy: 0.4863


In [10]:
# Evaluar rendimiento del modelo
model.evaluate(X_test, y_test)

7/7 [==============================] - 1s 93ms/step - loss: 144.8336 - accuracy: 0.5050


[144.83355712890625, 0.5049999952316284]

: 

# Conclusiones

# Bibliografía